# **MODEL CREATION AND EXPORT**

In [ ]:
import pandas as pd
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.similarities import MatrixSimilarity

class AnimeRecommender:
    def __init__(self, anime_df):
        self.df = anime_df.copy()
        self.documents = [doc.split() for doc in self.df['Synopsis']]
        self.dictionary = Dictionary(self.documents)
        self.corpus = [self.dictionary.doc2bow(doc) for doc in self.documents]
        self.tfidf = TfidfModel(self.corpus)
        self.index = MatrixSimilarity(self.tfidf[self.corpus])

    def recommend_anime(self, anime_name):
        idx = self.df[self.df['Name'] == anime_name].index[0]
        sims = self.index[self.tfidf[self.dictionary.doc2bow(self.documents[idx])]]
        sims = sorted(enumerate(sims), key=lambda item: -item[1])
        sim_indices = [i[0] for i in sims]
        return self.df['Name'].iloc[sim_indices[1:11]]  # Exclude the input anime itself

    def save_model(self, filepath):
        model_data = {
            "documents": self.documents,
            "dictionary": self.dictionary,
            "corpus": self.corpus,
            "tfidf_model": self.tfidf,
            "index": self.index,
        }
        pd.to_pickle(model_data, filepath)

# Load the dataset
anime_df = pd.read_csv(r"anime.csv")

# Create and export the model
recommender = AnimeRecommender(anime_df)
recommender.save_model("anime_recommender_model.pkl")
